In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib widget
matplotlib.rc('font', size=18)
default_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

import xarray as xr
import pandas as pd
from scipy import sparse

import sys
sys.path.append('../Climate-Learning/')

import general_purpose.uplotlib as uplt
import general_purpose.cartopy_plots as cplt
import general_purpose.utilities as ut

HOME = './'

In [ ]:
def l2(x, **kwargs):
    return np.sqrt(np.sum(x**2, **kwargs))

In [ ]:
lon = np.load('common/lon.npy')
lat = np.load('common/lat.npy')
W = sparse.load_npz('common/W.npz')

LON, LAT = np.meshgrid(lon,lat)

In [ ]:
X_std = np.load(f"common/r800y/fold_0/X_std.npy")
coslat = np.cos(lat*np.pi/180)
assert np.sum(X_std != 1) == W.shape[0]
aw = (np.ones_like(X_std).T * coslat).T
aw *= (X_std != 1)
aw /= np.sum(aw)
reshaper = ut.Reshaper(X_std != 1)

aw.shape

In [ ]:
y = 'r800y' # use the networks trained on 800 years of data

folder_GA = f'GA/{y}/0/'
folder_IINN = f'IINN/{y}/0/'

### Load projection patterns

In [ ]:
projs_GA = []
for fold in range(5):
    proj = np.load(f'{folder_GA}/fold_{fold}/proj.npy')
    norm = l2(proj*np.sqrt(aw))
    print(norm)
    projs_GA.append(proj/norm)

projs_GA = np.stack(projs_GA)

projs_IINN = []
signs_IINN = []
for fold in range(5):
    proj = np.load(f'{folder_IINN}/fold_{fold}/proj.npy')
    sign = -np.sign(np.mean(proj[...,-1])) # we want IINN projection patterns to have the proper sign, with negative values of soil moisture
    projs_IINN.append(proj/l2(proj*np.sqrt(aw))*sign)
    signs_IINN.append(sign)

projs_IINN = np.stack(projs_IINN)
signs_IINN = np.array(signs_IINN)

projs_GA.shape, projs_IINN.shape

### Load predictions

In [ ]:
# A_te = np.load('common/A_te.npy')
A_te = xr.open_dataarray('common/A_te.nc').data
f_tes_GA = np.stack([np.load(f'{folder_GA}/fold_{fold}/f_te.npy') for fold in range(5)])
f_tes_IINN = np.stack([np.load(f'{folder_IINN}/fold_{fold}/f_te.npy') for fold in range(5)])

A_pred_tes_GA = np.stack([np.load(f'{folder_GA}/fold_{fold}/Y_pred_te.npy') for fold in range(5)])
A_pred_tes_IINN = np.stack([np.load(f'{folder_IINN}/fold_{fold}/Y_pred_te.npy') for fold in range(5)])

# A_pred_tes_* contain mu and sigma. We turn them into ufloats for easier handling
A_pred_tes_GA = uplt.ufloatify(*A_pred_tes_GA.T).T
A_pred_tes_IINN = uplt.ufloatify(*A_pred_tes_IINN.T).T

A_te.shape, f_tes_GA.shape, f_tes_IINN.shape, A_pred_tes_GA.shape, A_pred_tes_IINN.shape

### Plot the projection patterns

In [ ]:
mfp_kwargs = dict(one_fig_layout=120, figsize=(10,5),
                  projections=[cplt.ccrs.Orthographic(central_latitude=90), cplt.ccrs.PlateCarree()],
                  extents=[None, (-5, 10, 39, 55)],
                  titles=['Geopotential height', 'Soil moisture'],
                 )
cmaps=['RdBu_r', 'BrBG']

#### Plot all folds

In [ ]:
for fold in range(5):
    _ = cplt.mfp(LON,LAT,projs_GA[fold], **mfp_kwargs, fig_num=8+fold, cmaps=cmaps)
    fig = _[0].get_figure()
    fig.suptitle(f'GA: fold {fold}')

In [ ]:
for fold in range(5):
    _ = cplt.mfp(LON,LAT,projs_IINN[fold], **mfp_kwargs, fig_num=8+fold, cmaps=cmaps)
    fig = _[0].get_figure()
    fig.suptitle(f'IINN: fold {fold}')

#### Plot only the best fold

In [ ]:
fold = 4 # fold 4 is the one with the best skill

mx = [7,15]

fig = cplt.mfp(LON,LAT,projs_GA[fold], mx=mx, **mfp_kwargs, fig_num=7, cmaps=cmaps)[0].get_figure()
fig.suptitle(r'$M_\mathrm{GA}$')

# fig.savefig(f'{HOME}/projection-pattern-GA.png', dpi=300)

fig = cplt.mfp(LON,LAT,projs_IINN[fold], mx=mx, **mfp_kwargs, fig_num=8, cmaps=cmaps)[0].get_figure()
fig.suptitle(r'$M_\mathrm{IINN}$')

# fig.savefig(f'{HOME}/projection-pattern-IINN.png', dpi=300)

### Plot the projected space

In [ ]:
fold = 4 # fold 4 is the one with the best skill

plt.close(3)
fig,axs = plt.subplots(1,2, figsize=(20,6), num=3)


axs[0].scatter(
    f_tes_GA[fold],
    A_te,
    marker='.',
    alpha=0.5, color='black', label='data'
)

isort = np.argsort(f_tes_GA[fold])

uplt.errorband(f_tes_GA[fold][isort], A_pred_tes_GA[fold][isort], color=default_colors[0], label='GA', ax=axs[0])

axs[0].set_xlabel(r'$M_\mathrm{GA}\cdot X$')
axs[0].set_ylabel('$A$ [K]')
axs[0].legend()

# fig.tight_layout()

# fig.savefig(f'{HOME}/projected-space-GA.pdf')


# plt.close(4)
# fig,ax = plt.subplots(figsize=(9,6), num=4)

isort = np.argsort(f_tes_IINN[fold])

axs[1].scatter(
    signs_IINN[fold]*f_tes_IINN[fold],
    A_te,
    marker='.',
    # '.',
    alpha=0.5, color='black', label='data',
)

uplt.errorband(signs_IINN[fold]*f_tes_IINN[fold][isort], A_pred_tes_IINN[fold][isort], color=default_colors[1], label='IINN')

plt.xlabel('$M_\mathrm{IINN}\cdot X$')
plt.ylabel('$A$ [K]')
plt.legend()

fig.tight_layout()

# fig.savefig(f'{HOME}/projected-space-GA-IINN.png', dpi=300)

### Put everything into a single figure

In [ ]:
kw = dict(mode='pcolormesh',
        greenwich=True,
        draw_gridlines=False, draw_labels=False,
         )

plt.close(1)
fig = plt.figure(num=1, figsize=(20, 10))


# plot geoplots
projs = np.concatenate([projs_GA[fold], projs_IINN[fold]], axis=-1)

for i in range(4):
    ax = fig.add_subplot(241 + i, projection=mfp_kwargs['projections'][i%2])

    _mx = mx[i%2]
    _norm = matplotlib.colors.TwoSlopeNorm(vcenter=0., vmin=-_mx, vmax=_mx)

    cplt.geo_plotter(ax, LON, LAT, projs[...,i], cmap=cmaps[i%2], norm=_norm, title=mfp_kwargs['titles'][i%2], **kw)

    if i%2:
        ax.set_extent(mfp_kwargs['extents'][1])


# plot projected space
axs = [fig.add_subplot(223), fig.add_subplot(224)]


axs[0].scatter(
    f_tes_GA[fold],
    A_te,
    marker='.',
    alpha=0.5, color='black', label='data'
)

isort = np.argsort(f_tes_GA[fold])

uplt.errorband(f_tes_GA[fold][isort], A_pred_tes_GA[fold][isort], color=default_colors[0], label='GA', ax=axs[0])

axs[0].set_xlabel(r'$M_\mathrm{GA}\cdot X$', fontdict=dict(size=20))
axs[0].set_ylabel('$A$ [K]', fontdict=dict(size=20))
axs[0].legend()


isort = np.argsort(f_tes_IINN[fold])

axs[1].scatter(
    signs_IINN[fold]*f_tes_IINN[fold],
    A_te,
    marker='.',
    # '.',
    alpha=0.5, color='black', label='data',
)

uplt.errorband(signs_IINN[fold]*f_tes_IINN[fold][isort], A_pred_tes_IINN[fold][isort], color=default_colors[1], label='IINN', ax=axs[1])

axs[1].set_xlabel(r'$M_\mathrm{IINN}\cdot X$', fontdict=dict(size=20))
axs[1].set_ylabel('$A$ [K]', fontdict=dict(size=20))
axs[1].legend()

# fig.suptitle(r'$M_\mathrm{GA}$' + ' '*95 + r'$M_\mathrm{IINN}$')
fig.suptitle(r'$M_\mathrm{GA}$' + ' '*77 + r'$M_\mathrm{IINN}$', fontsize='x-large')

fig.tight_layout(w_pad=0)

fig.savefig(f'{HOME}/GA-IINN.pdf')